In [40]:
import re
import os
import cv2
from tqdm import tqdm
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import pickle as pkl
with open("geometry_dirui_strip.pkl", 'rb') as f:
    geometry_strip = pkl.load(f)
with open("geometry_dirui_template_wide.pkl", 'rb') as f:
    geometry_template = pkl.load(f)
print geometry_strip.keys(), geometry_template.keys()

['coords_strip_mm', 'h_strip_mm', 'w_strip_mm'] ['w_template_mm', 'scale', 'coords_roi_mm', 'h_template_mm', 'h_roi_mm', 'w_roi_mm']


In [42]:
geometry_args = geometry_strip.copy()
geometry_args.update(geometry_template)
from processor_java import Processor
processor = Processor(**geometry_args)

In [43]:
def resize_to_720(image):
    scale = 720. / image.shape[1]
    return cv2.resize(image, (720, int(image.shape[0] * scale)))

In [45]:
path_in = "../../images/Photos_24_10_17/"
image_names = glob(path_in + "*/*.jpg")
path_out = "../../images/crops/24_10_17/"

In [46]:
def parse_id(name):
    return re.findall("[0-9]{8}", name)[0]

def parse_phone(name):
    tokens = name.split("/")
    for phone in ["LG", "Lenovo", "Xiaomi", "Pop"]:
        if phone in tokens:
            return phone
    return "Unknown"

In [49]:
def make_crops(image_names, path_out):
    
    for name in tqdm(image_names):
        image = resize_to_720(cv2.imread(name))
        id_ = parse_id(name)
        phone = parse_phone(name)
    
        try:
            os.makedirs("{}{}/{}".format(path_out, phone, id_))
        except:
            pass
    
        try:
            processor.fit(image)
            for key, crop in processor.colorbar.iteritems():
                cv2.imwrite("{}{}/{}/{}.png".format(path_out, phone, id_, key), crop)
            for key, crop in processor.palette.iteritems():
                cv2.imwrite("{}{}/{}/{}.png".format(path_out, phone, id_, key), crop)
        except RuntimeError as RE:
            print("Something went wrong on {} image:{}".format(name, RE.message))

In [50]:
make_crops(image_names, path_out)

100%|██████████| 92/92 [00:20<00:00,  5.92it/s]
